## netCDF: Importing netCDF files, extracting desired data and bringing it into a desired format

In this file I will itterativly try to load and understand the structure of the by anemos delivered netCDF wind data.

1. I am going to install and load all the necessary packages to work with and analyse netCDF data

In [1]:
import xarray as xr
import timeit
import pandas as pd
import numpy as np
# import numba as nb
import sys
def format_bytes(size):
    # 2**10 = 1024
    power = 2**10
    n = 0
    power_labels = {0 : '', 1: 'kilo', 2: 'mega', 3: 'giga', 4: 'tera'}
    while size > power:
        size /= power
        n += 1
    return size, power_labels[n]+'bytes'

import matplotlib.pyplot as plt

print(xr.__version__)

0.20.2


# 1. 10 minute data

Loading a netCDF file with the xarray package

In [3]:
#path = r"windData/wdirC.7L.2017-T.ts.nc"
year = 2017
test_data_path = f"windData/wspd.7L.{year}-T.ts.nc"
real_data_path = f"/uba/transfer/UBA-Windatlas/NC-Format/wspd.10L.{year}.nc"

start = timeit.default_timer()

data = xr.open_dataset(real_data_path, chunks={'time': 100}, engine='h5netcdf')
# for multiple files
#data = xr.open_mfdataset(real_data_path, chunks={'time': 100}, parallel=True, engine='h5netcdf')

stop = timeit.default_timer()

print(f'Loadingtme for netCDF-file: {stop - start}')
size = format_bytes(sys.getsizeof(data))
print(f"Size of loaded file: {size}")
data

Loadingtme for netCDF-file: 0.08367922000070394
Size of loaded file: (112, 'bytes')


<xarray.Dataset>
Dimensions:  (y: 310, x: 225, time: 52560, level: 10)
Coordinates:
    lat      (y, x) float32 dask.array<chunksize=(310, 225), meta=np.ndarray>
    lon      (y, x) float32 dask.array<chunksize=(310, 225), meta=np.ndarray>
  * time     (time) datetime64[ns] 2017-01-01 ... 2017-12-31T23:50:00
  * x        (x) int16 0 1 2 3 4 5 6 7 8 ... 216 217 218 219 220 221 222 223 224
  * y        (y) int16 0 1 2 3 4 5 6 7 8 ... 301 302 303 304 305 306 307 308 309
  * level    (level) float32 40.0 60.0 80.0 100.0 ... 170.0 200.0 250.0 300.0
Data variables:
    wspd     (time, level, y, x) float32 dask.array<chunksize=(100, 10, 310, 225), meta=np.ndarray>
Attributes: (12/130)
    TITLE:                           D3E5
    START_DATE:                      2011-07-01_00:00:00
    SIMULATION_START_DATE:           2010-01-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        [256]
    SOUTH-NORTH_GRID_DIMENSION:      [352]
    BOTTOM-TOP_GRID_DIMENSION:       [50]
    ...                              ...
    ISOILWATER:                      [14]
    history:                         Thu Aug  8 14:07:48 2019: ncrcat -7 -L 3...
    NCO:                             4.4.2
    title:                           D3E5
    nco_openmp_thread_number:        [1]
    actual_range:                    [100. 100.]

In [ ]:
type(data)

In [ ]:
data2d = data.sel(time=f"{year}-03-29T19:30", level=120)
data2d.to_array().plot(figsize=(10,12))


Defining Functions to get wspd (windspeed) at a given set of global coordinates, inside the local coordinate system, given by the anemos data.
Solution to filter lon, lat if they come as Data variables. ALL in numpy -> use numba to accelerate!

https://stackoverflow.com/questions/58758480/xarray-select-nearest-lat-lon-with-multi-dimension-coordinates

In [ ]:
#[ 40.,  60.,  80., 100., 130., 160., 200., 250., 300.]
#@nb.vectorize(nopython=True)
def findPoint(xarray, Point):
    abslat = np.abs(xarray.lat-Point["coor"][0])
    abslon = np.abs(xarray.lon-Point["coor"][1])
    c = np.maximum(abslon, abslat)
    
    if 'level' in xarray.dims:
        abslev = np.abs(xarray.level-Point["level"])
        nearestlevel = np.where(abslev == np.min(abslev))
        nearestlevel = xarray.level[nearestlevel]
        return np.where(c == np.min(c)), nearestlevel
    
    return np.where(c == np.min(c)), None

def getPointData(xarray, x, y, level=None, time=None):
    if level:
        return xarray.sel(x=x, y=y, level=level)
    
    return xarray.sel(x=x, y=y)

Timed example of applying the findPoint and getPointData function.

In [ ]:
point = {
    "level":131,
    "y":0,
    "x":0,
    "coor":[47.22,5.899],
    "nearestLevel":0,
    "time":pd.to_datetime(["2016-05-01"])
}

start = timeit.default_timer()

([yloc], [xloc]), nlevel  = findPoint(data, point)

mid = timeit.default_timer()

point_ds = getPointData(xarray=data, x=xloc, y=yloc, level=nlevel)

end = timeit.default_timer()

point["x"] = xloc
point["y"] = yloc

print(f'Nearest point coordinates: {point_ds.lon.values},{point_ds.lat.values}')
print(f'Nearest point: {point["x"]},{point["y"]}')
print(f'Time to find nearest point: {mid - start}')
print(f'Time to find get point data: {end - mid}')

#point_ds

In [ ]:
point_ds.wspd.plot(figsize=(22,4))

# 2. yearly average data

The averaged data contains wind statistics of each coordinate at a certain level. In 12 directional subclases, both the average windspeed and the dominant wind direction is given.

In [ ]:
def circularHisto(xarray, dataVariable:str, grid=False):
    '''
    To Do's:
    - Add colorbar
    '''
    radii = xarray[dataVariable].values
    N = radii.size
    theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)
    width = np.full((1, 12), 2 * np.pi / 13)[0]
    
    ax = plt.subplot(111, projection='polar')
    bars = ax.bar(theta, radii, bottom=0.0, width=width)
    # Use custom colors and opacity
    for r, bar in zip(radii, bars):
        bar.set_facecolor(plt.cm.viridis(r / radii.max()))
        bar.set_alpha(0.7)

    ax.set_theta_zero_location("N")
    if grid:
        ax.set_rticks(np.arange(0,radii.max(),2))
    else:
        ax.set_rticks([])
    
    ticks = ["N","NW","W","SW","S","SE","E","NE"]
    ax.set_xticklabels(ticks)
    
    if dataVariable == "wspd":
        ax.set_title("Average Windspeed [m/s] 2008 - 2017", pad=25)
        
    if dataVariable == "histo":
        ax.set_title("Distribution of wind directions 2008 - 2017", pad=25)
    plt.show()
    
def describingHisto(xarray):
    fig, axs = plt.subplots(1, 2)
    
    diags =("histo","wspd")
    
    for num, ax in enumerate(axs):
        ax = circularHisto(xarray, dataVariable=diags[num])

In [ ]:
path = r"windData/D-3km.E5.dirStats.140m.2008-2017.nc"

data120m = xr.open_dataset(path)
data120m

In [ ]:
point = {
    "level":114,
    "y":0,
    "x":0,
    "coor":[50,10],
    "time":pd.to_datetime(["2016-05-01"])
}

start = timeit.default_timer()

([yloc], [xloc]), nlevel  = findPoint(data120m, point)

mid = timeit.default_timer()

point_stat = getPointData(xarray=data120m, x=xloc, y=yloc)

end = timeit.default_timer()

point["x"] = xloc
point["y"] = yloc

print(f'Nearest point: {point["x"]},{point["y"]}')
print(f'Nearest point coordinates: {point_stat.lon.values},{point_stat.lat.values}')
print(f'Time to find nearest point: {mid - start}')
print(f'Time to find get point data: {end - mid}')

point_stat

In [ ]:
circularHisto(point_stat, dataVariable="histo")
circularHisto(point_stat, dataVariable="wspd", grid=True)

In [ ]:
import numpy as np
import numba
from numba import cuda, f8, uint8

n = 20
dk = 0.00001

In [ ]:

def frange(n=n, dk =dk):
    dk = dk
    X = np.arange(dk, n, dk)
    outerSum = 0
    for i in range(X.shape[0]):
        k = X[i]
        outerSum += k

%timeit frange()


In [ ]:
@numba.jit(nopython=True)
def farange(n=n, dk =dk):
    dk = dk
    #X = np.arange(dk, n, dk)
    outerSum = 0
    for i in np.arange(dk, n, dk):
        outerSum += i

%timeit farange()

In [ ]:
@numba.jit(nopython=True)
def nbrange(n=n, dk =dk):
    dk = dk
    X = np.arange(dk, n, dk)
    outerSum = 0
    for i in range(X.shape[0]):
        k = X[i]
        outerSum += k

%timeit nbrange()

In [ ]:
@cuda.jit(argtypes=[f8, uint8])
def cudarange(n=n, dk =dk):
    dk = dk
    X = np.arange(dk, n, dk)
    outerSum = 0
    for i in range(X.shape[0]):
        k = X[i]
        outerSum += k

%timeit cudarange()